In [ ]:
!pip install openai==0.28

In [ ]:
import requests
import openai
import json

# Set up Asana API Configuration
ASANA_API_KEY = "Your_Asana_Key"
BASE_URL = "Asana_Base_URL"

HEADERS = {
    "Authorization": f"Bearer {ASANA_API_KEY}",
    "Content-Type": "application/json"
}


# Session state to track current project and tasks
session_state = {
    "current_project": None,  # Stores the active project name
    "tasks": {}  # Stores tasks by project: {"project_name": ["task1", "task2", ...]}
}


# Function to fetch a project ID by name
def get_project_id_by_name(workspace_id, project_name):
    url = f"{BASE_URL}/projects?workspace={workspace_id}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        projects = response.json().get('data', [])
        for project in projects:
            if project.get('name').lower() == project_name.lower():
                return project.get('gid')
        return None
    else:
        print(f"Error fetching projects: {response.status_code} {response.text}")
        return None

# Function to fetch a task ID by name
def get_task_id_by_name(project_id, task_name):
    url = f"{BASE_URL}/projects/{project_id}/tasks"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        tasks = response.json().get('data', [])
        for task in tasks:
            if task.get('name').lower() == task_name.lower():
                return task.get('gid')
        return None
    else:
        print(f"Error fetching tasks: {response.status_code} {response.text}")
        return None

# Function to create a task
def create_task(name, workspace_id, project_id, assignee=None, due_date=None):
    url = f"{BASE_URL}/tasks"
    data = {
        "data": {
            "name": name,
            "workspace": workspace_id,
            "projects": [project_id],
            "assignee": assignee,
            "due_on": due_date
        }
    }
    print(f"Request URL: {url}")
    print(f"Request Data: {json.dumps(data, indent=2)}")

    response = requests.post(url, headers=HEADERS, json=data)
    print(f"Response Status Code: {response.status_code}")
    print(f"Response Body: {response.text}")

    if response.status_code == 201:
        print("Task created successfully:", response.json())
        return response.json()
    else:
        print(f"Error creating task: {response.status_code} {response.text}")
        return None

# Function to list tasks using project name
def list_tasks_by_project_name(workspace_id, project_name):
    project_id = get_project_id_by_name(workspace_id, project_name)
    if not project_id:
        return f"Project '{project_name}' not found in workspace {workspace_id}."
    return list_tasks(project_id)

# Function to list tasks
def list_tasks(project_id):
    url = f"{BASE_URL}/projects/{project_id}/tasks"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        tasks = response.json().get('data', [])
        if tasks:
            return "\n".join([f"- {task['name']} (ID: {task['gid']})" for task in tasks])
        else:
            return "No tasks found."
    else:
        return f"Error fetching tasks: {response.json()}"


# Function to delete a task by name
def delete_task_by_name(workspace_id, project_name, task_name):
    # Get project ID
    project_id = get_project_id_by_name(workspace_id, project_name)
    if not project_id:
        return f"Project '{project_name}' not found."

    # Get task ID
    task_id = get_task_id_by_name(project_id, task_name)
    if not task_id:
        return f"Task '{task_name}' not found in project '{project_name}'."

    # Delete the task
    deletion_response = delete_task(task_id)
    if "successfully" in deletion_response:
        return f"Task '{task_name}' deleted successfully."

    # Delete the task
    return delete_task(task_id)

# Function to delete a task
def delete_task(task_id):
    url = f"{BASE_URL}/tasks/{task_id}"
    response = requests.delete(url, headers=HEADERS)
    print(f"Request URL: {url}")
    print(f"Response Status Code: {response.status_code}")
    print(f"Response Body: {response.text}")

    if response.status_code == 200:
        return f"Task deleted successfully."
    else:
        return f"Error deleting task: {response.status_code} {response.text}"


# Function to mark a task as complete using task name
def complete_task_by_name(workspace_id, project_name, task_name):
    project_id = get_project_id_by_name(workspace_id, project_name)
    if not project_id:
        return f"Project '{project_name}' not found in workspace {workspace_id}."
    task_id = get_task_id_by_name(project_id, task_name)
    if not task_id:
        return f"Task '{task_name}' not found in project '{project_name}'."
    return complete_task(task_id)

# Function to mark a task as complete
def complete_task(task_id):
    url = f"{BASE_URL}/tasks/{task_id}"
    data = {"data": {"completed": True}}
    response = requests.put(url, headers=HEADERS, json=data)
    print(f"Request URL: {url}")
    print(f"Request Data: {json.dumps(data, indent=2)}")
    print(f"Response Status Code: {response.status_code}")
    print(f"Response Body: {response.text}")

    if response.status_code == 200:
        print("Task marked as complete successfully.")
        return "Task marked as complete successfully."
    else:
        return f"Error marking task as complete: {response.status_code} {response.text}"

# Configure OpenAI API key
openai.api_key = "You_OpenAI_Key"

def process_command(command, workspace_id):
    messages = [
        {"role": "system", "content": "You are an assistant for managing Asana tasks."},
        {"role": "user", "content": f"""
        Based on the user command, determine whether to create a task, list tasks in a project, mark a task as complete, or delete a task.
        Assume the context of the current project if one exists; otherwise, prompt for the project name.
        User command: "{command}"

        Return a JSON object with keys: action (create, list, complete, delete), project_name, task_name, assignee, and due_date. Leave irrelevant keys empty.
        """}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0.7,
            max_tokens=200
        )
        response_text = response["choices"][0]["message"]["content"].strip()
        result = json.loads(response_text)
    except Exception as e:
        return f"Error processing command: {str(e)}"

    # Extract relevant fields
    action = result.get("action")
    project_name = result.get("project_name")
    task_name = result.get("task_name")
    assignee = result.get("assignee")
    due_date = result.get("due_date")

    # Use the current project if none is provided
    if not project_name and session_state["current_project"]:
        project_name = session_state["current_project"]

    # Update the current project for the session
    if action in ["create", "list", "complete", "delete"] and project_name:
        session_state["current_project"] = project_name

    if action == "create":
        if not task_name:
            task_name = input("What is the task name? ").strip()
        if not assignee:
            assignee = input("Who should be assigned to this task (email)? ").strip() or None
        if not due_date:
            due_date = input("What is the due date for this task (YYYY-MM-DD)? ").strip() or None
        project_id = get_project_id_by_name(workspace_id, project_name)
        if not project_id:
            return f"Project '{project_name}' not found."
        return create_task(task_name, workspace_id, project_id, assignee, due_date)

    elif action == "list":
        return list_tasks_by_project_name(workspace_id, project_name)

    elif action == "complete":
        if not task_name:
            task_name = input("What is the task name to mark as complete? ").strip()
        return complete_task_by_name(workspace_id, project_name, task_name)

    elif action == "delete":
        if not task_name:
            task_name = input("What is the task name to delete? ").strip()
        return delete_task_by_name(workspace_id, project_name, task_name)

    else:
        return "Invalid action specified."


def repl():
    workspace_id = "your_workspace_ID"  # Replace with your workspace ID
    print("Welcome to the AI-Powered Asana Task Manager")
    while True:
        if session_state["current_project"]:
            print(f"\nCurrent project: {session_state['current_project']}")
        else:
            print("\nNo active project. Specify a project in your next command.")

        command = input("Enter your command (type 'quit' to exit): ").strip()
        if command.lower() == "quit":
            print("Goodbye! Resetting session.")
            session_state["current_project"] = None
            session_state["tasks"] = {}
            break
        print(process_command(command, workspace_id))


if __name__ == "__main__":
    repl()


Welcome to the AI-Powered Asana Task Manager

No active project. Specify a project in your next command.
Request URL: https://app.asana.com/api/1.0/tasks
Request Data: {
  "data": {
    "name": "101",
    "workspace": "1209163165395352",
    "projects": [
      "1209185361815857"
    ],
    "assignee": null,
    "due_on": null
  }
}
Response Status Code: 201
Response Body: {"data":{"gid":"1209199029478822","projects":[{"gid":"1209185361815857","resource_type":"project","name":"Moveworks"}],"due_on":null,"num_likes":0,"num_hearts":0,"workspace":{"gid":"1209163165395352","resource_type":"workspace","name":"My workspace"},"tags":[],"start_on":null,"resource_subtype":"default_task","due_at":null,"notes":"","completed_at":null,"assignee_status":"upcoming","completed":false,"actual_time_minutes":0,"created_at":"2025-01-21T01:20:58.253Z","assignee":null,"resource_type":"task","name":"101","modified_at":"2025-01-21T01:20:59.092Z","start_at":null,"parent":null,"likes":[],"liked":false,"hearts":